# Robust Lot Sizing

Consider the problem of Lot Sizing, where you need to decide how much to produce or order in each time period to meet demand and minimize costs. This involves planning for a fixed period with known demand and various costs such as setup, production, holding inventory, and handling backorders. The main goal is to create a production plan that meets demand efficiently. In its robust form, this problem also accounts for uncertainties in production yields, which can vary within a certain range.

In the example taken from [1, Section 3.3] we consider a single item multi period uncapacitated lot sizing problem with backorder and production yield that determine the quantity to produce in each time period of the finite planning horizon $T$.



The problem we want to solve is the uncertain linear optimization problem of the form:

$$
\begin{array}{ll}
\text{minimize} & \sum_{t = 1}^T\left(s_t^Ty_t + v_t^Tx_t + {h'}_t\right) \\
\text{subject to} & h'_t \geq h_t\left( p_t^Tx_t - d_t\right) \quad t =1,\dots,T \quad \forall p_t \in \mathcal{U}\\ 
                  & h'_t \geq -b_t\left( p_t^Tx_t - d_t\right) \quad t =1,\dots,T \quad \forall p_t \in \mathcal{U}\\
                  & x_t \leq {m_t}^T{y_t} \quad t =1,\dots,T \\
                  &  x \geq \mathbf{0}\\
                  & {h'} \geq \mathbf{0}
\end{array}
$$

where $s$ denotes the setup cost, $h$ denotes the inventory holding cost , $b$ denotes the backorder cost $b$, $x$ denotes the lot size to be produced, and $d$ denotes the demand for each time period in $T$. $h'$ is independently defined for each period as the highest cost between the worst inventory cost and the worst backorder cost under the uncertainty set. The model contains the decision variables $X$, the lot size to be produced, and the setup decision $Y$. The strictly positive uncertain production yield is defined as $p$. $m$ denotes the upper bound on production quantity.

To solve this problem, we first import the required packages.

In [31]:
import numpy as np
import lropt
import cvxpy as cp

We start by defining the relevant constants and creating variables. 

In [32]:
np.random.seed(1)
T = 5 
RHO = 2
s = np.random.rand(T)  
v = np.random.rand(T)  
h = np.random.rand(T)  
b = np.random.rand(T)  
d = np.random.randint(10, 20, size=T)  

# Large positive number for setup constraint
m = np.full(T, 1000)
x = cp.Variable(T, nonneg=True)
y = cp.Variable(T, boolean=True)  
v = cp.Variable(T, nonneg=True) 

In the following we define the parameter $p$ as an uncertain parameter that belongs a Budget Uncertainty set. The budget uncertainty is defined as:
$$
        \mathcal{U}_{\text{budg}} = \{z \ | \ \|z \|_\infty \le \rho_1,
        \|z \|_1 \leq \rho_2\}
$$

In [33]:
p = lropt.UncertainParameter(T, uncertainty_set = lropt.Budget(rho1 = RHO, rho2= RHO), nonneg = True)

We define our objective and constraints.

In [34]:
objective = cp.Minimize(cp.sum(s @ y + v @ x + v))
constraints = [
v >= h * cp.sum(p @ x - d),
v >= (-b) * cp.sum(p @ x - d),
x <= m@y
]

Finally, we solve the problem and get the optimal value.

In [35]:
prob = lropt.RobustProblem(objective, constraints)
prob.solve()

/Users/mj5676/Desktop/miniconda3/envs/lropt_v3/lib/python3.12/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [36]:
formatted_values = [f"{value:.3E}" for value in np.array(v.value)]
print(f"The robust optimal values for the total cost using Budget uncertainty are {', '.join(formatted_values)}")

The robust optimal values for the total cost using Budget uncertainty are 6.034E+01, 3.756E+01, 5.028E+01, 1.263E+01, 1.783E+01


## References

1. Metzker, P., Thevenin, S., Adulyasak, Y., & Dolgui, A. (2023). Robust optimization for lot-sizing problems under yield uncertainty. https://doi.org/10.1016/j.cor.2022.106025